In [ ]:
!pip install cohere==5.5.0 \
    langchain==0.2.0 \
    langchain_core==0.2.0 \
    langchain_openai==0.2.0 \
    langchain_experimental==0.2.0 \
    numpy==1.26.4 \
    pandas==2.2.2 \
    pymongo==4.6.1 \
    pytz==2024.1 \
    qdrant_client==1.9.1 \
    sentence_transformers==2.2.2 \
    streamlit==1.33.0 \
    langchain-community==0.2.0 \
    faiss-cpu==1.8.0 \
    pdf2image==1.17.0 \
    git-lfs==1.6 \
    file-chat-input==0.0.2 \
    streamlit-float==0.3.4 \
    poppler-utils==0.1.0


ERROR: Could not find a version that satisfies the requirement langchain_experimental==0.2.0 (from versions: 0.0.1rc1, 0.0.1rc2, 0.0.1rc3, 0.0.1rc4, 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15, 0.0.16, 0.0.17, 0.0.18, 0.0.19, 0.0.20, 0.0.21, 0.0.22, 0.0.23, 0.0.24, 0.0.25, 0.0.27, 0.0.28, 0.0.29, 0.0.30, 0.0.31, 0.0.32, 0.0.33, 0.0.34, 0.0.35, 0.0.36, 0.0.37, 0.0.38, 0.0.39, 0.0.40, 0.0.41, 0.0.42, 0.0.43, 0.0.44, 0.0.45, 0.0.46, 0.0.47, 0.0.48, 0.0.49, 0.0.50, 0.0.51, 0.0.52, 0.0.53, 0.0.54, 0.0.55, 0.0.56, 0.0.57, 0.0.58, 0.0.59, 0.0.60, 0.0.61, 0.0.62, 0.0.63, 0.0.64, 0.0.65, 0.3.0.dev1, 0.3.0, 0.3.1, 0.3.1.post1, 0.3.2)
ERROR: No matching distribution found for langchain_experimental==0.2.0


In [ ]:
!pip install langchain_openai langchain_experimental langchain_core

In [ ]:
!pip install langchain
!pip install langchain_openai langchain_experimental
!pip install langchainhub
!pip install langchain_together
!pip -q install --upgrade together
!pip install fastembed
!pip install qdrant-client
!pip install sentence_transformers
!pip install langchain-cohere
!pip install einops
!pip install PyPDF2
!pip install pypdf
!pip install -U langchain-qdrant

In [ ]:
import os
os.environ["TOGETHER_API_KEY"] = "35bf344aa6b5cdb53fed22ee813932b7661d3e7e25cec5798f9c35118c1fc4ce"
os.environ["COHERE_API_KEY"] = 'xxe3X6u8vcTFJgJ8Pc7CfLezwpQiATQcUB56VIUp'

TOP_K = 5
MAX_DOCS_FOR_CONTEXT = 5

In [ ]:
from operator import itemgetter
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from qdrant_client.http import models
from langchain_core.documents.base import Document
from langchain.load import dumps, loads
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.retrievers import BaseRetriever
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain_cohere.llms import Cohere
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from sentence_transformers.util import cos_sim
import PyPDF2
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_qdrant import Qdrant
import re
import nltk
import cohere
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pathlib import Path
import uuid
import time

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
embedding_model = CohereEmbeddings(model="embed-english-v3.0", cohere_api_key=os.environ["COHERE_API_KEY"], user_agent="my-app")
semantic_chunker_embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
qdrant_client = QdrantClient(
    url="https://fe88bef8-9e9c-425b-8070-a1057b923fcb.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="HX3YCIr_9NE3lEGfH6lqRE53NurR9sBE_GuWoIzhE2u5rT8AsOWmzw",
)

QDRANT_URL = "https://fe88bef8-9e9c-425b-8070-a1057b923fcb.europe-west3-0.gcp.cloud.qdrant.io:6333"
QDRANT_API_KEY = "HX3YCIr_9NE3lEGfH6lqRE53NurR9sBE_GuWoIzhE2u5rT8AsOWmzw"

In [ ]:
qdrant_client = QdrantClient(
    url="https://d85d4513-7b30-4ecc-9181-15977cab8cd4.europe-west3-0.gcp.cloud.qdrant.io",
    api_key="pLDWhzmelSHihIuiiwYBs0WhZVqHMTNEaHFVeKmAIAewFJ8WMDZoCA",
     https=True,
    prefer_grpc=True
)

QDRANT_URL = "https://d85d4513-7b30-4ecc-9181-15977cab8cd4.europe-west3-0.gcp.cloud.qdrant.io"
QDRANT_API_KEY = "pLDWhzmelSHihIuiiwYBs0WhZVqHMTNEaHFVeKmAIAewFJ8WMDZoCA"

In [ ]:
def preprocess_text(text):
    # 1. Lowercasing
    text = text.lower()

    # 2. Remove unwanted characters (punctuation, special characters)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # 3. Tokenization
    tokens = word_tokenize(text)

    # 4. Stopword Removal
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Lemmatization
    # lemmatizer = WordNetLemmatizer()
    # tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Return the preprocessed tokens (or join them into a single string if needed)
    return " ".join(tokens)


In [ ]:
def create_QDrant_collection():
		embeddings = CohereEmbeddings(model = "embed-english-v3.0", user_agent="my-app")
		qdrant_client.recreate_collection(
		collection_name="asapp_hackathon",
		vectors_config = models.VectorParams(size=1024, distance=models.Distance.COSINE),
	)

create_QDrant_collection()

<ipython-input-27-7557f4548dea>:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


In [ ]:
# def upload_chunks_to_QDrant(documents):
#   records_to_upload = []
#   for idx, chunk in enumerate(documents):
#       content = chunk.page_content
#       vector = embedding_model.embed_query(content)

#       record = models.PointStruct(
#           id=idx,
#           vector=vector,
#           payload={"page_content": content}
#       )
#       records_to_upload.append(record)

#   qdrant_client.upload_points(
#       collection_name="asapp_hackathon",
#       points=records_to_upload
#   )
def upload_chunks_to_QDrant(documents):
    records_to_upload = []
    for chunk in documents:
        content = chunk.page_content
        vector = embedding_model.embed_query(content)

        # Create a unique UUID for each chunk
        unique_id = str(uuid.uuid4())

        record = models.PointStruct(
            id=unique_id,
            vector=vector,
            payload={"page_content": content}
        )
        records_to_upload.append(record)

    qdrant_client.upload_points(
        collection_name="asapp_hackathon",
        points=records_to_upload
    )


In [ ]:
# def upload_pdf_to_QDrant(pdf_path):
#     semantic_chunker = SemanticChunker(
#         semantic_chunker_embed_model, breakpoint_threshold_type="percentile"
#     )

#     pdf_file_path = pdf_path
#     pdf_file = open(pdf_file_path, 'rb')
#     pdf_reader = PyPDF2.PdfReader(pdf_file)

#     text = ''
#     for page_num in range(len(pdf_reader.pages)):
#         page = pdf_reader.pages[page_num]
#         text = page.extract_text()
#         text = preprocess_text(text) # Preprocesing
#         #print(text)
#         documents = semantic_chunker.create_documents([text])
#         upload_chunks_to_QDrant(documents)
#         #print(documents)

#     pdf_file.close()
#     print(f"Uploaded {pdf_path} to Qdrant.")

def upload_pdf_to_QDrant(pdf_path):
    semantic_chunker = SemanticChunker(
        semantic_chunker_embed_model, breakpoint_threshold_type="percentile"
    )

    pdf_file_path = pdf_path
    pdf_file = open(pdf_file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    text = ''
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        text = preprocess_text(text)  # Preprocessing

        # Create and upload document chunks
        documents = semantic_chunker.create_documents([text])
        upload_chunks_to_QDrant(documents)

    pdf_file.close()
    print(f"Uploaded {pdf_path} to Qdrant.")

In [ ]:
def create_QDrant_collection():
		embeddings = CohereEmbeddings(model = "embed-english-v3.0", user_agent="my-app")
		qdrant_client.recreate_collection(
		collection_name="asapp_hackathon_1",
		vectors_config = models.VectorParams(size=1024, distance=models.Distance.COSINE),
	)

create_QDrant_collection()

<ipython-input-52-a3578257b18b>:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


In [ ]:
def upload_multiple_pdfs_to_qdrant(pdf_paths, delay=1):
    for pdf_path in pdf_paths:
        # Load and chunk the PDF
        loader = PyPDFLoader(pdf_path)
        pages = loader.load_and_split()

        # Use Qdrant's from_documents to handle embedding and upload
        qdrant = Qdrant.from_documents(
            documents=pages,
            embedding=embedding_model,
            collection_name="asapp_hackathon",
            url=QDRANT_URL,
            prefer_grpc=True,
            api_key=QDRANT_API_KEY
        )

        print(f"Uploaded {pdf_path} to Qdrant.")

        # Wait to avoid exceeding the rate limit
        time.sleep(delay)

In [ ]:
upload_multiple_pdfs_to_qdrant(files[25:30], delay=3)  # Adjust delay as needed

TooManyRequestsError: status_code: 429, body: data=None message='trial token rate limit exceeded, limit is 100000 tokens per minute'

In [ ]:
def upload_multiple_pdfs_to_QDrant(pdf_paths):
    for pdf_path in pdf_paths:
        print(f"Processing {pdf_path}...")
        upload_pdf_to_QDrant(pdf_path)
        print(f"Uploaded {pdf_path} to Qdrant.")

In [ ]:
upload_multiple_pdfs_to_QDrant(files)

Processing /content/2306.11695.pdf...
Uploaded /content/2306.11695.pdf to Qdrant.
Uploaded /content/2306.11695.pdf to Qdrant.
Processing /content/1710.05941.pdf...
Uploaded /content/1710.05941.pdf to Qdrant.
Uploaded /content/1710.05941.pdf to Qdrant.
Processing /content/2110.15343.pdf...
Uploaded /content/2110.15343.pdf to Qdrant.
Uploaded /content/2110.15343.pdf to Qdrant.
Processing /content/2310.06825.pdf...
Uploaded /content/2310.06825.pdf to Qdrant.
Uploaded /content/2310.06825.pdf to Qdrant.
Processing /content/2306.17806.pdf...
Uploaded /content/2306.17806.pdf to Qdrant.
Uploaded /content/2306.17806.pdf to Qdrant.
Processing /content/2306.12929.pdf...


Uploaded /content/2306.12929.pdf to Qdrant.
Uploaded /content/2306.12929.pdf to Qdrant.
Processing /content/ICML03-094.pdf...
Uploaded /content/ICML03-094.pdf to Qdrant.
Uploaded /content/ICML03-094.pdf to Qdrant.
Processing /content/2301.00774.pdf...
Uploaded /content/2301.00774.pdf to Qdrant.
Uploaded /content/2301.00774.pdf to Qdrant.
Processing /content/2005.14165.pdf...
Uploaded /content/2005.14165.pdf to Qdrant.
Uploaded /content/2005.14165.pdf to Qdrant.
Processing /content/Adaptive-mixtures-of-local-experts.pdf...
Uploaded /content/Adaptive-mixtures-of-local-experts.pdf to Qdrant.
Uploaded /content/Adaptive-mixtures-of-local-experts.pdf to Qdrant.
Processing /content/1907.01470.pdf...
Uploaded /content/1907.01470.pdf to Qdrant.
Uploaded /content/1907.01470.pdf to Qdrant.
Processing /content/2305.18290.pdf...
Uploaded /content/2305.18290.pdf to Qdrant.
Uploaded /content/2305.18290.pdf to Qdrant.
Processing /content/2310.20707.pdf...
Uploaded /content/2310.20707.pdf to Qdrant.
Up

Uploaded /content/2301.13688.pdf to Qdrant.
Uploaded /content/2301.13688.pdf to Qdrant.
Processing /content/2211.09110.pdf...


TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"

In [ ]:
import os

def list_files_with_extension(directory, extension):
    # Normalize the extension
    extension = extension.lower()
    files_with_extension = []

    # Iterate through the directory
    for filename in os.listdir(directory):
        # Check if the file has the desired extension
        if filename.lower().endswith(extension):
            # Get the full path and append it to the list
            full_path = os.path.join(directory, filename)
            files_with_extension.append(full_path)

    return files_with_extension

# Example usage
directory_path = '/content'  # Replace with your directory path
file_extension = '.pdf'  # Replace with the desired extension
files = list_files_with_extension(directory_path, file_extension)
print(files)


['/content/2306.11695.pdf', '/content/1710.05941.pdf', '/content/2110.15343.pdf', '/content/2310.06825.pdf', '/content/2306.17806.pdf', '/content/2306.12929.pdf', '/content/ICML03-094.pdf', '/content/2301.00774.pdf', '/content/2005.14165.pdf', '/content/Adaptive-mixtures-of-local-experts.pdf', '/content/1907.01470.pdf', '/content/2305.18290.pdf', '/content/2310.20707.pdf', '/content/2207.00112.pdf', '/content/2211.05102.pdf', '/content/2203.02155.pdf', '/content/2104.09864.pdf', '/content/2307.13304.pdf', '/content/2001.08361.pdf', '/content/2302.10866.pdf', '/content/2305.07185.pdf', '/content/2306.07629.pdf', '/content/10000000_662098952474184_2584067087619170692_n.pdf', '/content/2109.06243.pdf', '/content/2301.13688.pdf', '/content/2211.09110.pdf', '/content/palm2techreport.pdf', '/content/1706.03762.pdf', '/content/2002.05202.pdf', '/content/333078981_693988129081760_4712707815225756708_n.pdf', '/content/2302.13971.pdf', '/content/1810.04805.pdf', '/content/2308.10792.pdf', '/cont

In [ ]:
upload_multiple_pdfs_to_qdrant(files)

TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"

In [ ]:
upload_pdf_to_QDrant("/content/2307.09288.pdf")

In [ ]:
query = preprocess_text("""We present a new supervised learning procedure for systems composed of many separate
 networks, each of which learns to handle a subset of the complete set of training cases.
 The new procedure can be viewed either as a modular version of a multilayer supervised
 network, or as an associative version of competitive learning. It therefore provides a new
 link between these two apparently different approaches. We demonstrate that the learning
 procedure divides up a vowel discrimination task into appropriate subtasks, each of which
 can be solved by a very simple expert network""")

    # Generate embedding vector for the query
query_vector = embedding_model.embed_query(query)

# Perform the search in Qdrant
results = qdrant_client.search(
    collection_name='asapp_hackathon',
    query_vector=query_vector,
    limit=TOP_K,  # Number of top results to retrieve
)


In [ ]:
results

[ScoredPoint(id='c0225b46-21b4-4785-841d-2c9da6fe82a7', version=10, score=0.5661770105361938, payload={'metadata': {'page': 5, 'source': '/content/Adaptive-mixtures-of-local-experts.pdf'}, 'page_content': 'arenowafunction ofthecurren tinput vector andarerepresen tedbyactivit ylevelsrather\nthanweights.Inaddition, weuseagating networkwhichallowsthemixing proportions of\ntheexpertstobedetermined bytheinput vector. Thisgivesusasystem ofcompetinglocal\nexpertswiththeerrorfunction de\x0cned inequation 3.Wecould alsointroduceamechanism\ntoallowtheinput vectortodynamically determine thecovariance matrix forthedistribution\nde\x0cned byeachexpertnetwork,butwehavenotyetexperimen tedwiththispossibilit y.\n3Application tomulti-sp eakervowelrecognition\nThemixture ofexpertsmodelwasevaluated onaspeakerindependen t,four-class, vowel\ndiscrimination problem. Thedataconsisted ofthe\x0crstandsecond forman tsofthevowels[i],\n[I],[a],and[A](usually denoted [\x03])from75speakers(males, females andchildren

In [ ]:

file_names = [os.path.basename(file_path) for file_path in files[:25]]
file_names

['2306.11695.pdf',
 '1710.05941.pdf',
 '2110.15343.pdf',
 '2310.06825.pdf',
 '2306.17806.pdf',
 '2306.12929.pdf',
 'ICML03-094.pdf',
 '2301.00774.pdf',
 '2005.14165.pdf',
 'Adaptive-mixtures-of-local-experts.pdf',
 '1907.01470.pdf',
 '2305.18290.pdf',
 '2310.20707.pdf',
 '2207.00112.pdf',
 '2211.05102.pdf',
 '2203.02155.pdf',
 '2104.09864.pdf',
 '2307.13304.pdf',
 '2001.08361.pdf',
 '2302.10866.pdf',
 '2305.07185.pdf',
 '2306.07629.pdf',
 '10000000_662098952474184_2584067087619170692_n.pdf',
 '2109.06243.pdf',
 '2301.13688.pdf']